In [110]:
import os
import pdfplumber
from collections import defaultdict

def detect_columns(text_blocks, page_width):
    """Gruppiere Textblöcke basierend auf ihrer horizontalen Position (Spalten)."""
    columns = defaultdict(list)
    threshold = page_width * 0.15  # Threshold, um Spalten zu erkennen

    for block in text_blocks:
        x_center = (block['x0'] + block['x1']) / 2

        assigned = False
        for col_x in columns:
            if abs(col_x - x_center) < threshold:
                columns[col_x].append(block)
                assigned = True
                break
        
        if not assigned:
            columns[x_center].append(block)

    return columns

def extract_text_from_column(column_blocks):
    """Sortiere die Blöcke innerhalb einer Spalte und extrahiere den Text."""
    sorted_blocks = sorted(column_blocks, key=lambda block: (block['top'], block['x0']))
    return ' '.join([block['text'] for block in sorted_blocks])

def extract_text_from_page_g1(page):
    """Extrahiere den Text aus einer Seite für PDFs in g1, behalte dabei das Layout."""
    text_blocks = page.extract_words()
    if not text_blocks:
        return ""

    columns = detect_columns(text_blocks, page.width)
    full_text = []

    for col_x, blocks in sorted(columns.items()):
        column_text = extract_text_from_column(blocks)
        full_text.append(column_text)

    return "\n\n".join(full_text)

def extract_text_from_page_g2(page):
    """Alternative Methode zur Textextraktion für PDFs in g2."""
    # Beispielhafte einfache Extraktion, die besser auf das Layout abgestimmt ist
    # Dies kann durch Methoden wie pdfminer.six, die textorientierter sind, ergänzt werden.
    text_blocks = page.extract_words()
    if not text_blocks:
        return ""
    
    full_text = [block['text'] for block in sorted(text_blocks, key=lambda block: (block['top'], block['x0']))]
    
    return ' '.join(full_text)

def extract_text_from_pdfs(pdf_folder_path, extraction_function):
    documents = []

    for filename in os.listdir(pdf_folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder_path, filename)

            with pdfplumber.open(pdf_path) as pdf:
                full_text = []

                for page in pdf.pages:
                    page_text = extraction_function(page)
                    full_text.append(page_text)

                documents.append("\n\n".join(full_text))

    return documents

# Pfade zu den PDF-Ordnern
pdf_folder_path_g1 = os.path.join('data', 'pdf', 'g1')
pdf_folder_path_g2 = os.path.join('data', 'pdf', 'g2')

# Extrahiere die Texte aus den PDFs in g1
documents_g1 = extract_text_from_pdfs(pdf_folder_path_g1, extract_text_from_page_g1)

# Extrahiere die Texte aus den PDFs in g2
documents_g2 = extract_text_from_pdfs(pdf_folder_path_g2, extract_text_from_page_g2)

# Kombiniere alle Dokumente
documents = documents_g1 + documents_g2

# Beispielhafte Ausgabe des ersten Dokuments
print(documents[2] if documents else "Keine Dokumente gefunden")

In die Glascontainer gehören: ausschließlich restentleerte Glasverpackungen. Die Glasverpackungen sind nach den Farben Weiß, Braun und Grün zu sortieren und in die dafür vorgesehenen Container zu werfen. Nicht zuordenbare Farben, wie z. B. blaues Glas, kommen zum Grünglas. Deckel müssen nicht abgeschraubt werden. Zum Beispiel: · alle nicht bepfandeten Glasflaschen z. B. für Wein, Sekt, Spirituosen, Essig, Öl, Milch, Frucht- und Gemüse- säfte sowie Fruchtnektare · F lakons aus Glas, z. B. Parfümflaschen · Marmeladen-, Gurken- und Senfgläser · sonstiges Verpackungsglas für Obst, Soßen, Suppen, Gemüse GRÜN- GLAS WEISS- BRAUN- GLAS GLAS Nicht in die Glascontainer gehören: sämtliche Abfälle, auch aus Glas, die keine Verpackungen sind. Zum Beispiel: · Auflaufformen · hitzebeständiges Glas · Steingutflaschen · Autolampen · Isolierglas · Teller, Tassen · Autoscheiben · Kaffeekannen · Trinkgläser · Batterien · K amin- und Ofenglas · Bio- und Restabfall · Keramik · Bleiglas · Leuchtstoffröhren ·

In [108]:
import re

def clean_text(documents):
    """Durchläuft die Liste der Dokumente und bereinigt die Texte."""
    cleaned_documents = []
    
    # Liste unerwünschter Zeichen oder Muster
    unwanted_patterns = [
        r"",     # Entferne das Symbol 
        r"",     # Beispiel für ein weiteres unerwünschtes Symbol
        r"(\b\w+\s*-\s*\w+\b)",  # Entferne Worte mit unvollständigen Endungen (z.B. "Glasfl aschen")
        r"\n+",   # Entferne überflüssige Zeilenumbrüche
        r"\s{2,}",# Entferne überflüssige Leerzeichen
        r"\s*:\s*",  # Entferne überflüssige Doppelpunkte
        r"\b\w{1,2}\b",  # Entferne isolierte kurze Wörter (z.B. "fi" in "finden")
        r"[^A-Za-z0-9äöüÄÖÜß.,;:!?()\-'\s]"  # Entferne nicht-alphanumerische Zeichen
    ]

    # Regex-Muster kombinieren
    combined_pattern = re.compile("|".join(unwanted_patterns))

    for document in documents:
        # Entferne unerwünschte Zeichen und Muster
        cleaned_text = combined_pattern.sub(" ", document)
        # Entferne mehrfach aufeinanderfolgende Leerzeichen, um den Text zu bereinigen
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        cleaned_documents.append(cleaned_text)
    
    return cleaned_documents

# Verwende die bestehende documents-Liste aus dem vorherigen Skript
cleaned_documents = clean_text(documents)

# Beispielhafte Ausgabe eines bereinigten Dokuments
print(cleaned_documents[2])

die Gelbe Tonne den Gelben Sack gehören ausschließlich gebrauchte und restentleerte Verpackungen, die nicht aus Papier, Pappe, Karton oder Glas sind. Verpackungsbestandteile bitte voneinander trennen. Ausspülen ist nicht notwendig. Zum Beispiel -, Menüschalen von Kunststoffdeckel Fertiggerichten Arzneimittelblister Butterfolie Getränkekartons Müsliriegelfolie Joghurtbecher Nudeltüten Einkaufstüten sowie Reinigungsmittelflaschen Gemüsebeutel Nachfüllbeutel aus Kunststoff . . für Waschmittel, Flüssigseife Eisverpackungen oder Fruchtpüree üllmaterial von Versand- Senftuben verpackungen aus Kunststoff, Shampooflaschen wie . . Luftpolsterfolie Spraydosen oder Schaumstoff Soßentüten Konservendosen Tierfutterdosen Kronkorken Zahnpastatuben Kunststoffschalen für Lebensmittel Nicht die Gelbe Tonne den Gelben Sack gehören Verpackungen aus Papier, Pappe, Karton und Glas sowie sämtliche Abfälle, die keine Verpackungen sind. Zum Beispiel Schuhe Altkleider Gummi Strumpfhosen Batterien und Akkus Holz

In [109]:
import re
import spacy

# Lade das deutsche spaCy-Modell
nlp = spacy.load('de_core_news_sm')

def preprocess_text_with_advanced_cleaning(documents):
    """Bereinigt die Dokumente, entfernt unnötige Stopwörter und normalisiert den Text."""
    custom_stopwords = nlp.Defaults.stop_words.difference({'nicht', 'kein', 'wichtig'})

    preprocessed_documents = []

    for document in documents:
        text = document.lower()
        text = re.sub(r'\b(?:[\w\-]+\.)+[\w]{2,4}(?:/[\w\-.?%&=]*)?\b', '', text)
        text = re.sub(r'\b[\w\.\-]+@[\w\.\-]+\.\w{2,4}\b', '', text)
        text = re.sub(r'\b(?:www|servicetelefon|servicetelefax|services|frankfurt|main|weidenbornstraße|liebfrauenberg)\b', '', text)
        text = re.sub(r'[^a-zäöüß\s]', '', text)

        doc = nlp(text)
        filtered_tokens = [token.text for token in doc if token.text not in custom_stopwords and len(token.text) > 2]

        # Entferne unzusammenhängende Zeichenfolgen und extrem kurze Wörter
        text = ' '.join(filtered_tokens)
        text = re.sub(r'\b\w{1,3}\b', '', text)

        # Entferne mehrfach aufeinanderfolgende gleiche Wörter
        text = re.sub(r'\b(\w+)\s+\1\b', r'\1', text)

        # Entferne weiterhin sinnlose Textfragmente
        text = re.sub(r'\b[a-zäöüß]{2,3}\b\s+\b[a-zäöüß]{2,3}\b', '', text)

        preprocessed_documents.append(text.strip())

    return preprocessed_documents

# Bereinige die Dokumente weiter
preprocessed_documents = preprocess_text_with_advanced_cleaning(cleaned_documents)

# Beispielhafte Ausgabe eines bereinigten Dokuments
print(preprocessed_documents[2])

gelbe tonne gelben sack gehören ausschließlich gebrauchte restentleerte verpackungen nicht papier pappe karton glas verpackungsbestandteile bitte voneinander trennen ausspülen nicht notwendig menüschalen kunststoffdeckel fertiggerichten arzneimittelblister butterfolie getränkekartons müsliriegelfolie joghurtbecher nudeltüten einkaufstüten reinigungsmittelflaschen gemüsebeutel nachfüllbeutel kunststoff waschmittel flüssigseife eisverpackungen fruchtpüree üllmaterial versand senftuben verpackungen kunststoff shampooflaschen luftpolsterfolie spraydosen schaumstoff soßentüten konservendosen tierfutterdosen kronkorken zahnpastatuben kunststoffschalen lebensmittel nicht gelbe tonne gelben sack gehören verpackungen papier pappe karton glas sämtliche abfälle verpackungen schuhe altkleider gummi strumpfhosen batterien akkus holzwolle styroporreste behälterglas hygieneartikel dämmplatten blechgeschirr katzenstreu tapetenreste  disketten keramikprodukte windeln druckerpatronen kinderspielzeug zah